## MAIN

In [ ]:
!pip install visualtorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 8.9 MB/s eta 0:00:00


In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import time
import random
import torch
import visualtorch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from torch.utils.data import DataLoader
from IPython.display import HTML
from torchvision import utils, datasets, transforms
import pandas as pd
from math import sqrt

# Inputs

In [ ]:
# @title Main Input {"run":"auto","vertical-output":true,"display-mode":"both"}
workers = 2  # @param {"type":"slider","min":0,"max":16,"step":1}



# Number of workers for dataloader    # It depends on your CPU model. It represents the maximum available threads of your CPU. A large number may cause an error. It is recommended to start from 1 and try to increase it gradually.
                      #Need Change

# Batch size during training
#default 365   #Need Change  It is an important hyperparameter. If your memory or GPU memory is overflowing, please reduce this value.
batch_size = 300  # @param {"type":"number"}


# Spatial size of training images. All images will be resized to this
#   size using a transformer.  #Need Change  Change the number if you want another resolution for the outputs
image_size = 128 # @param {"type":"number"}

# Number of channels in the training images. For color images this is 3. There is no need to change it for the single channel weather data
nc = 1  # @param {"type":"number"}

# Size of z latent vector (i.e. size of generator input)
# Very important! The complex graph should be set larger, and the simple graph with good consistency should be set smaller, otherwise the D network will converge quickly, and G cannot be further optimized, resulting in failure
nz = 10000     # @param {"type":"number"}                        #Need Change   Can try to change If the training effect is not good.

# Size of feature maps in generator  测试过32，收敛效果不好
ngf = 64       # @param {"type":"number"}                     #Need Change   Can try to change If the training effect is not good.

# Size of feature maps in discriminator  测试过32，收敛效果不好
ndf = 64       # @param {"type":"number"}                     #Need Change   Can try to change If the training effect is not good.

# Number of training epochs
num_epochs = 50  # @param {"type":"number"} #default50               #Need Change    If this value is large, the training time will increase
                                                # More iterations doesn't mean the result is necessarily better.
                                                # Find the number of iterations at which D and G converge to 0.5

# Learning rate for optimizers
#0.0002
lr = 0.0002 # @param {"type":"number"} #0.0002                   #Need Change   Can try to change If the training effect is not good.

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5 # @param {"type":"number"}                          #Need Change   Can try to change If the training effect is not good.

# Number of GPUs available.
ngpu = 1     # @param {"type":"number"}                         #Need Change   Use 0 for CPU mode. Use 1 for GPU mode.

# Number of pics in line for repersenting the results (useless here)
nrows = 30   # @param {"type":"number"}


### ONLY FOR COLAB

In [ ]:
from google.colab import files
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#!unzip /content/drive/MyDrive/dati_trunc_expanded_float.zip
!unzip /content/drive/MyDrive/2023-07-26H15-00_2024-12-07H14_00.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_11430.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_7324.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_6514.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_6940.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_4961.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_7074.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_10224.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_5123.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_8018.tiff  
  inflating: content/Dati/2023-07-26H15-00_2024-12-07H14_00/train/255data/image_11698.tiff  
  inflating: content/Dati/2

In [ ]:
# @title  {"run":"auto"}
dataroot = "/content/content/Dati/2012_2024_trunc_exp_f/train" # @param {"type":"string"}
data_valid = '/content/content/Dati/2012_2024_trunc_exp_f/test'     # @param {"type":"string"}

In [ ]:
!unzip /content/drive/MyDrive/china.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: data/TrainDB/255data/DB2009057D.tif  
  inflating: data/TrainDB/255data/DB2009057N.tif  
  inflating: data/TrainDB/255data/DB2009058D.tif  
  inflating: data/TrainDB/255data/DB2009058N.tif  
  inflating: data/TrainDB/255data/DB2009059D.tif  
  inflating: data/TrainDB/255data/DB2009059N.tif  
  inflating: data/TrainDB/255data/DB2009060D.tif  
  inflating: data/TrainDB/255data/DB2009060N.tif  
  inflating: data/TrainDB/255data/DB2009061D.tif  
  inflating: data/TrainDB/255data/DB2009061N.tif  
  inflating: data/TrainDB/255data/DB2009062D.tif  
  inflating: data/TrainDB/255data/DB2009062N.tif  
  inflating: data/TrainDB/255data/DB2009063D.tif  
  inflating: data/TrainDB/255data/DB2009063N.tif  
  inflating: data/TrainDB/255data/DB2009064D.tif  
  inflating: data/TrainDB/255data/DB2009064N.tif  
  inflating: data/TrainDB/255data/DB2009065D.tif  
  inflating: data/TrainDB/255data/DB2009065N.tif  
  inflating: data/TrainDB/255dat

In [ ]:
# @title  {"run":"auto"}
dataroot = "data/TrainDB" # @param {"type":"string"}
data_valid = 'data/TestDB'     # @param {"type":"string"}

In [ ]:
!unrar x  /content/drive/MyDrive/DatiTesi/data.rar

check dtype

In [ ]:
from PIL import Image
mine =Image.open('/content/content/Dati/2012_2024_trunc_exp_f/train/255data/image_10000.tiff')
china=Image.open('/content/data/TrainDB/255data/DB2000001D.tif')
imgm = np.array(mine)
imgc = np.array(china)
print(imgm.shape)
print(imgc.shape)

(128, 128)
(128, 128)


In [ ]:
imgm

array([[128.62909 , 128.62909 , 126.26835 , ..., 120.83963 , 120.491005,
        120.491005],
       [128.62909 , 128.62909 , 126.26835 , ..., 120.83963 , 120.491005,
        120.491005],
       [130.22781 , 130.22781 , 127.936806, ..., 121.77596 , 122.01005 ,
        122.01005 ],
       ...,
       [173.80194 , 173.80194 , 176.75536 , ..., 163.95058 , 163.76628 ,
        163.76628 ],
       [173.85173 , 173.85173 , 176.86493 , ..., 164.25937 , 164.09999 ,
        164.09999 ],
       [173.85173 , 173.85173 , 176.86493 , ..., 164.25937 , 164.09999 ,
        164.09999 ]], dtype=float32)

In [ ]:
imgc

array([[129.82051 , 123.16497 , 118.217964, ...,  97.231514,  96.2625  ,
         96.56849 ],
       [109.62447 , 114.87747 , 122.37448 , ...,  96.2625  ,  96.87447 ,
         94.55397 ],
       [114.87747 , 124.03204 , 121.7625  , ...,  96.87447 ,  96.186005,
         96.083984],
       ...,
       [125.23047 , 124.873505, 126.45449 , ..., 110.49147 , 107.813995,
        107.96699 ],
       [129.15749 , 126.45449 , 120.20704 , ..., 108.91048 , 107.12553 ,
        126.19947 ],
       [127.24499 , 120.20704 , 117.04498 , ..., 139.17896 , 175.23601 ,
        112.302025]], dtype=float32)

### LOCAL

In [ ]:
# Root directory for dataset
dataroot = "2012_2024_pad/2016_2024"   # @param {"type":"string"}    #Need Change
data_valid = '2012_2024_pad/2012_2015'   # @param {"type":"string"}             #Need Change

# **SPERIMENTALE**

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.image = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
        )
        self.prior = nn.Sequential(
          nn.ConvTranspose2d(1, ngf * 8, 4, 1, 0, bias=False),
          nn.BatchNorm2d(ngf * 8),
          nn.ReLU(True),

        )
        self.main = nn.Sequential(
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 32 x 32
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, image, prior):
        image = self.image(image)
        prior = self.prior(prior)
        incat = torch.cat((image, prior), dim=1)
        return self.main(incat)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.image = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
            # state size. (ndf) x 32 x 32
        )
        self.prior = nn.Sequential(
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
            # state size. (ndf) x 32 x 32
        )
        self.main = nn.Sequential(
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 16 x 16
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 8 x 8
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*16) x 4 x 4
            nn.Conv2d(ndf * 16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # state size. (1) x 1 x 1
        )

    def forward(self, image, prior):
        image = self.image(image)
        prior = self.prior(prior)
        incat = torch.cat((image, prior), dim=1)
        return self.main(incat)


# Generator

In [ ]:
#ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)


# Generator Code
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.image = nn.Sequential(
            # input is nz, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
        )
        self.prior = nn.Sequential(
            # state size. 1 x 1 x 1
            nn.ConvTranspose2d(1, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True)
            # state size. (ngf*8) x 4 x 4
        )
        self.main = nn.Sequential(
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d( ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf * 2) x 32 x 32
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 128 x 128
        )

    def forward(self, image, prior):
        image = self.image(image)
        prior = self.prior(prior)
        #将两个张量（image, prior）拼接在一起, dim=1 means按维数1（列）拼接，即并排放一起
        incat = torch.cat((image, prior), dim=1)
        return self.main(incat)


# Discriminator

In [ ]:
# Discriminator Code

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.image = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
            # state size. (ndf) x 64 x 64
        )
        self.prior = nn.Sequential(
            # input is 1 x 128 x 128
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
            # state size. (ndf) x 64 x 64
        )
        self.main = nn.Sequential(
            # input is (ndf *2) x 64 x 64
            nn.Conv2d(ndf *2, ndf *4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf * 4) x 32 x 32
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf * 8) x 16 x 16
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*16) x 8 x 8
            nn.Conv2d(ndf *16, ndf * 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 32),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*32) x 4 x 4
            nn.Conv2d(ndf * 32, 1, 4, 1, 0, bias=False),
            # state size. (1) x 1 x 1
            nn.Sigmoid()
        )

    def forward(self, image, prior):
        image = self.image(image)
        prior = self.prior(prior)
        incat = torch.cat((image, prior), dim=1)
        return self.main(incat)



In [ ]:

if __name__ == '__main__':
# Set random seed for reproducibility
  manualSeed = 512
#manualSeed = random.randint(1, 10000) # use if you want new results
  print("Random Seed: ", manualSeed)
  random.seed(manualSeed)
  torch.manual_seed(manualSeed)
  torch.cuda.manual_seed(manualSeed)
# For the same manualSeed, the following torch.rand numbers are fixed

Random Seed:  512


# Data

In [ ]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
# transforms.ToTensor() Divide by 255 into (0,1)
# transforms.Normalize((0.5), (0.5)) means from (0,1) to (-1,1)
dataset = dset.ImageFolder(root=dataroot,
                            transform=transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5), (0.5)),
                            ]))
print(f'Total Size of Dataset for Train: {len(dataset)}')

# Create the dataset for Validation
dataset_valid = dset.ImageFolder(root=data_valid,
                            transform=transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5), (0.5)),
                            ]))
print(f'Total Size of Dataset for Test: {len(dataset_valid)}')
valid_num = len(dataset_valid)

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
#print("dataloader for train finished")
dataloader_valid = torch.utils.data.DataLoader(dataset=dataset_valid, batch_size=valid_num, shuffle=True, num_workers=workers)
#print("dataloader for validation finished")
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print("device to run on is", device)


Total Size of Dataset for Train: 9000
Total Size of Dataset for Test: 3000
device to run on is cuda:0


# Read the Test data   

In [ ]:
# Inizializzazione
valid_means = []  # Lista per salvare le medie (test set mean matrix tensor)
valid_num = len(dataloader_valid.dataset)  # Numero di esempi nel dataset
valid_label_zero = torch.zeros(valid_num, 1, 1, 1).to(device)  # Output tensor di dimensione 1*1*1 per ogni esempio

# Elaborazione del dataloader
for i, data_batch in enumerate(dataloader_valid):
    print(f"Batch {i+1}/{len(dataloader_valid)}")  # Informazioni sul batch corrente
    real_data_batch = data_batch[0].to(device)  # Ottieni i dati del batch
    batch_size = real_data_batch.size(0)  # Dimensione del batch corrente

    # Calcola la media per ogni immagine nel batch
    for m in range(batch_size):
        avg_value = torch.mean(real_data_batch[m])  # Media dei valori per un'immagine
        valid_label_zero[i * batch_size + m][0] = avg_value  # Salva nel tensor delle label
        valid_means.append(avg_value.item())  # Salva anche come numero scalare per analisi futura


Batch 1/1


In [ ]:

# There may be some confusion here for Validation and Test. For unsupervised classification, there is no validation during training. Validation is Testing here.

valid_label = []                             #Test set mean matrix Tensor
for i, data_valid_r in enumerate(dataloader_valid, 0):
    print(i)
    real_valid = data_valid_r[0].to(device)
#print("len of dataloader_valid is", len(dataloader_valid))      #1
valid_size = len(real_valid)
valid_mean = [0 for i in range(valid_size)]           #1D array of means for the test set
#print("number of real data used for Validation:", valid_size)                       #3654
valid_label_zero = torch.zeros(valid_num, 1, 1, 1).to(device)   #First, generate valid_num 1*1*1 Tensors with values of 0 (the test data is only used for the generator, so it is 1*1*1 instead of 1*128*128)
for m in range(valid_num):
    #print("m =", m)
    ones_labePrior = torch.ones(1, 1).to(device)
    Avg = torch.mean(real_valid[m])
    #print("Avg =", Avg)
    valid_label_zero[m][0]= ones_labePrior * Avg
    Avg_num = Avg.cpu()
    Avg_num = Avg_num.item()
    #print("Avg_num=", Avg_num)
    valid_mean [m] = Avg_num
    #print("valid_mean [m]=", valid_mean [m])
valid_label = valid_label_zero.to(device)

##################################################


0


In [ ]:

#Define the image mean correlation index calculation function in the verification period
def corrcoef(x, y):  # x,y are mean arraies
    n = len(x)
    #print("len(x)",n)
    # Sum
    sum1 = sum(x)
    #print("sum(x)",sum1)
    sum2 = sum(y)
    #print("sum(y)",sum2)
    sum3 = 0    #sum of squared errors
    sum4 = 0    #sum of absolute values of errors
    sum5 = 0    #sum of absolute percentage errors
    # To calculate the sum of products
    multiplyofxy = np.multiply(x, y)
    sumofxy = 0
    for m in range (n):
        sumofxy = sumofxy + multiplyofxy [m]
        #Calculate the sum of squared errors
        sum3 = sum3 + (x[m]-y[m])*(x[m]-y[m])
        sum4 = sum4 + abs(x[m]-y[m])
        sum5 = sum5 + abs((x[m]-y[m])/x[m])
    #print("multiply(x, y)",sumofxy)
    #print("sum3=",sum3,"sum4=",sum4)
    rmse = sqrt(sum3/n)
    mae = sum4/n
    #print("rmse=",rmse,"mae=",mae)
    mape  = sum5/n *100  # Unit is %
    #print("mape=",mape)
    # Calculate the sum of squares
    sumofx2 = sum([pow(i, 2) for i in x])
    sumofy2 = sum([pow(j, 2) for j in y])
    num = sumofxy - (float(sum1) * float(sum2) / n)
    # Calculating the Pearson Correlation Coefficient
    den = sqrt((sumofx2 - float(sum1 ** 2) / n) * (sumofy2 - float(sum2 ** 2) / n))
    #print("num=",num)
    #print("den=",den)
    #print("num / den=",num / den)
    return num / den, rmse, mae, mape
# Returns the Pearson correlation coefficient, root mean square error (RMSE), mean absolute error (MAE), mean absolute percentage error (MAPE) in turn

# Define tensor correlation coefficient calculation function
def tensor_corrcoef(a, b):   #a,b are tensors  (num, band, X, Y)
    time1 = time.time()
    num_pic = a.size(0)     #num
    m = a.size(2)       #The value of image dimension 1
    n = a.size(3)       #The value of image dimension 2
    #print("num_pic m n are",num_pic, m, n)
    r2 = 0
    rmse = 0
    mae = 0
    mape = 0
    for p in range (num_pic):
        x = [0 for w in range(m*n)]
        y = [0 for w in range(m*n)]
        t = 0
        for q in range (m):
            for r in range (n):
                x[t] = (a[p][0][q][r].item() * 0.5 + 0.5)*100 - 50  # to be centigrade
                y[t] = (b[p][0][q][r].item() * 0.5 + 0.5)*100 - 50
                t = t + 1
        #print("x is",len(x), x)
        #print("y is",len(y), y)
        s = len(x)
        # Calculate sum
        sum1 = sum(x)
        #print("sum(x)",sum1)
        sum2 = sum(y)
        #print("sum(y)",sum2)
        # Computes the sum of products
        multiplyofxy = np.multiply(x, y)
        sumofxy = 0
        sum3 = 0
        sum4 = 0
        sum5 = 0
        for r in range (s):
            sumofxy = sumofxy + multiplyofxy [r]
            sum3 = sum3 + (x[r]-y[r])*(x[r]-y[r])
            sum4 = sum4 + abs(x[r]-y[r])
            sum5 = sum5 + abs((x[r]-y[r])/x[r])
        #print("multiply(x, y)",sumofxy)
        rmse = sqrt(sum3/s) + rmse
        mae = sum4/s + mae
        #print("rmse=",rmse,"mae=",mae)
        mape  = sum5/s *100 + mape  # Unit is %
        # Calculate the sum of squares
        sumofx2 = sum([pow(i, 2) for i in x])
        sumofy2 = sum([pow(j, 2) for j in y])
        num = sumofxy - (float(sum1) * float(sum2) / s)
        # Calculating the Pearson Correlation Coefficient
        den = sqrt((sumofx2 - float(sum1 ** 2) / s) * (sumofy2 - float(sum2 ** 2) / s))
        #print("num=",num)
        #print("den=",den)
        if (p + 1) % 100 == 0:
            time2 = time.time()
            print("Accounting r2 for validation pics {} have been finished ".format(p + 1))
            print("Time used", round(time2 - time1,1), "s")
            print("Estimated time remaining:", round((time2 - time1) / (p + 1) * (num_pic + 1), 1), "s")
        r2 = num / den + r2
    return r2 / (num_pic + 1), rmse / (num_pic + 1), mae / (num_pic + 1), mape / (num_pic + 1)


# Implementation

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv') != -1:
      nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif classname.find('BatchNorm') != -1:
      nn.init.normal_(m.weight.data, 1.0, 0.02)
      nn.init.constant_(m.bias.data, 0)

In [ ]:
######################################################################
# Now, we can instantiate the generator and apply the ``weights_init``
# function. Check out the printed model to see how the generator object is
# structured.
# First instantiate the generator and use the initialized weights (``weights_init``) to generate results

# Create the generator
netG = Generator(ngpu).to(device)
print("The generator has been created")
# to(device) is to determine whether to use CPU or GPU, and device has been defined as a parameter in the above code

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
  netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.02 （defined by "def weights_init(m)" above）.
netG.apply(weights_init)
print("The init weights have been applied")
# Print the model
print("Net G is", netG)


######################################################################
# Now, as with the generator, we can create the discriminator, apply the
# ``weights_init`` function, and print the model’s structure.
# Instantiate the discriminator, using the ``weights_init`` parameter

# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)


######################################################################
# Loss Functions and Optimizers
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# With :math:`D` and :math:`G` setup, we can specify how they learn
# through the loss functions and optimizers. We will use the Binary Cross
# Entropy loss
# (`BCELoss <https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss>`__)
# function which is defined in PyTorch as:
#
# .. math:: \ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad l_n = - \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right]
#
# Notice how this function provides the calculation of both log components
# in the objective function (i.e. :math:`log(D(x))` and
# :math:`log(1-D(G(z)))`). We can specify what part of the BCE equation to
# use with the :math:`y` input. This is accomplished in the training loop
# which is coming up soon, but it is important to understand how we can
# choose which component we wish to calculate just by changing :math:`y`
# (i.e. GT labels).
#
# Next, we define our real label as 1 and the fake label as 0. These
# labels will be used when calculating the losses of :math:`D` and
# :math:`G`, and this is also the convention used in the original GAN
# paper. Finally, we set up two separate optimizers, one for :math:`D` and
# one for :math:`G`. As specified in the DCGAN paper, both are Adam
# optimizers with learning rate 0.0002 and Beta1 = 0.5. For keeping track
# of the generator’s learning progression, we will generate a fixed batch
# of latent vectors that are drawn from a Gaussian distribution
# (i.e. fixed_noise) . In the training loop, we will periodically input
# this fixed_noise into :math:`G`, and over the iterations we will see
# images form out of the noise.
#

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize the progression of the generator
# 64 below is the batch size（注释原来的代码，已经用batch_size代替了64）
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device) #[batch, channel, height, width]
fixed_prior  = torch.randn(batch_size, 1, 1, 1, device=device) #[batch, channel, height, width]
# fixed_noise is only used to substitute the output results of the generator to compare the output results of each generation of models. It does not mean that this set of noises is always used in the model training process.
# The noise used for training is totally random!!!

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


The generator has been created
The init weights have been applied
Net G is Generator(
  (image): Sequential(
    (0): ConvTranspose2d(10000, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (prior): Sequential(
    (0): ConvTranspose2d(1, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (main): Sequential(
    (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (

# VISUALIZE MODELS

# Training

In [ ]:
# Finally, now that we have all of the parts of the GAN framework defined,
# we can train it. Be mindful that training GANs is somewhat of an art
# form, as incorrect hyperparameter settings lead to mode collapse with
# little explanation of what went wrong. Here, we will closely follow
# Algorithm 1 from Goodfellow’s paper, while abiding by some of the best
# practices shown in `ganhacks <https://github.com/soumith/ganhacks>`__.
# Namely, we will “construct different mini-batches for real and fake”
# images, and also adjust G’s objective function to maximize
# :math:`logD(G(z))`. Training is split up into two main parts. Part 1
# updates the Discriminator and Part 2 updates the Generator.
#
# **Part 1 - Train the Discriminator**
#
# Recall, the goal of training the discriminator is to maximize the
# probability of correctly classifying a given input as real or fake. In
# terms of Goodfellow, we wish to “update the discriminator by ascending
# its stochastic gradient”. Practically, we want to maximize
# :math:`log(D(x)) + log(1-D(G(z)))`. Due to the separate mini-batch
# suggestion from ganhacks, we will calculate this in two steps. First, we
# will construct a batch of real samples from the training set, forward
# pass through :math:`D`, calculate the loss (:math:`log(D(x))`), then
# calculate the gradients in a backward pass. Secondly, we will construct
# a batch of fake samples with the current generator, forward pass this
# batch through :math:`D`, calculate the loss (:math:`log(1-D(G(z)))`),
# and *accumulate* the gradients with a backward pass. Now, with the
# gradients accumulated from both the all-real and all-fake batches, we
# call a step of the Discriminator’s optimizer.

# **Part 2 - Train the Generator**
#
# As stated in the original paper, we want to train the Generator by
# minimizing :math:`log(1-D(G(z)))` in an effort to generate better fakes.
# As mentioned, this was shown by Goodfellow to not provide sufficient
# gradients, especially early in the learning process. As a fix, we
# instead wish to maximize :math:`log(D(G(z)))`. In the code we accomplish
# this by: classifying the Generator output from Part 1 with the
# Discriminator, computing G’s loss *using real labels as GT*, computing
# G’s gradients in a backward pass, and finally updating G’s parameters
# with an optimizer step. It may seem counter-intuitive to use the real
# labels as GT labels for the loss function, but this allows us to use the
# :math:`log(x)` part of the BCELoss (rather than the :math:`log(1-x)`
# part) which is exactly what we want.


# Finally, we will do some statistic reporting and at the end of each
# epoch we will push our fixed_noise batch through the generator to
# visually track the progress of G’s training. The training statistics
# reported are:
#
# -  **Loss_D** - discriminator loss calculated as the sum of losses for
#    the all real and all fake batches (:math:`log(D(x)) + log(1 - D(G(z)))`).
# -  **Loss_G** - generator loss calculated as :math:`log(D(G(z)))`
# -  **D(x)** - the average output (across the batch) of the discriminator
#    for the all real batch. This should start close to 1 then
#    theoretically converge to 0.5 when G gets better. Think about why
#    this is.
# -  **D(G(z))** - average discriminator outputs for the all fake batch.
#    The first number is before D is updated and the second number is
#    after D is updated. These numbers should start near 0 and converge to
#    0.5 as G gets better. Think about why this is.
#
# **Note:** This step might take a while, depending on how many epochs you
# run and if you removed some data from the dataset.
#
# Training Loop

# Lists to keep track of progress
#  img_list = []
G_losses = []
D_losses = []
D_x_list = []
D_z_list = []
loss_tep = 10
iters = 0

print("Starting Training Loop...(This may take a long time)")
# For each epoch
beg_time = time.time()
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        #print(i)
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()

        # Format batch
        real_cpu = data[0].to(device) #data[0] is the first dimension data of a data group in dataloader (there is no data[1] for pictures)，
                                                    # The format of data[0] is (batch size, number of channels, X dimension, Y dimension)
        b_size = real_cpu.size(0)        #data[0] The number of data in the first dimension, here should be equal to the batch size

        label_real = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        #torch.full means to create a 1-dimensional tensor with a batch size value of real_label
        label_fake = torch.full((b_size,), fake_label, dtype=torch.float, device=device)

    #How to define two priors (conditional inputs) for the calculation of netG and netD
        G_label_zero = torch.zeros(b_size, 1, 1, 1).to(device)
        for m in range(b_size):
            ones_labeG = torch.ones(1, 1).to(device)
            Avg = torch.mean(real_cpu[m])
            #The mean value of all the grids of a real picture m in the batch size (to be input as a conditional number)
            #print("avg is", Avg)
            G_label_zero[m][0]= ones_labeG * Avg
            #print("G_label_zero is", G_label_zero[m][0])
        G_label = G_label_zero.to(device)
        #print("G_label is", G_label.size())
        # state size. batch size x 1 x 1 x 1

        D_label_zero = torch.zeros(b_size, 1, 128, 128).to(device)
        for n in range(b_size):
            ones_labeD = torch.ones(128, 128).to(device)
            Avg = torch.mean(real_cpu[n])
            #Avg = Avg.cpu().numpy()
            #print("avg is", Avg)
            D_label_zero[n]= ones_labeD * Avg
        D_label = D_label_zero.to(device)
        #print("D_label is", D_label.size())
        # state size. batch size x 1 x 128 x 128
        #In a batch processing, G_label and D_label use the same group, which is calculated from the real image of the batch, instead of using random numbers every time

        # Forward pass real batch through D
        output = netD(real_cpu, D_label).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label_real)
        #errD_real is the effect of the discriminator on whether the real image is true (the distance calculation between the output result output and the real image label label_real, the larger the result, the better), which is equivalent to log(D(x))
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()
        #D_x is the mean value of the judgment result of the discriminator on the real image

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)  #[batch, channel, height, width]
        # Generate fake image batch with G
        fake = netG(noise, G_label)

        #Fake is some fake image generated using generator (based on noise and G_label)
        #label.fill_(fake_label) Replace the value of label with the value of fake_label, replacing the original value of real_label
        # Classify all fake batch with D
        output = netD(fake.detach(), D_label).view(-1)
        #output is the output of netD (a number)
        #detach() is used to cut off the backpropagation of some branches
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label_fake)
        #errD_fake is the discrimination effect of the discriminator on whether the fake picture is fake (calculate the distance between the output result output and the fake picture label label_fake, the bigger the result, the better), equivalent to log(1 - D(G(z)))
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        #D_G_z1 is the mean value of the judgment result of the discriminator for the fake image
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        print(errD_real.item())
        print(errD_fake.item())
        #print(errD.item())

        #errD is the loss function of the discriminator, which is equivalent to log(D(x)) + log(1 - D(G(z)))
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        #label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        # Use the updated D to discriminate the fake image generated by the previous G, and get the loss
        output = netD(fake, D_label).view(-1)
        #output是更新后的netD的输出结果（一个数）
        # Calculate G's loss based on this output
        errG = criterion(output, label_real)
        #errG is the loss function of the generator, which is the effect of the updated discriminator on discriminating whether the fake image is real, indicating the effect of the generator (calculation of the distance between the output result output and the fake image label label_real, the larger the result, the better)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        #D_G_z2 is the average value of the judgment result of the updated discriminator on the fake image
        # Update G
        optimizerG.step()


        # Output training stats
        end_time = time.time()
        run_time = round(end_time-beg_time,2)

        if i % 4 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f \tUsedTime:%.2f s'
                    % (epoch + 1, num_epochs, i + 1, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2, run_time))
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        # Save D(X) and D(G(z)) for plotting later Each batch counts a loss
        D_x_list.append(D_x)
        D_z_list.append(D_G_z2)
        # Save the Best Model
        #if errG < loss_tep:
        #    torch.save(netG.state_dict(), 'models/model-{}-{}.pt'.format(epoch + 1,i + 1))
        #    loss_tep = errG

        # Check how the generator is doing by saving G's output on fixed_noise
        #if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
        #    with torch.no_grad():
        #         fake = netG(fixed_noise,fixed_prior).detach().cpu()
        #    img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
        #iters += 1
        #Each generation of calculation is completed, and the model structure at that time is saved
    torch.save(netG.state_dict(), 'models/model-{}.pt'.format(epoch + 1))
    print("Trains of epoch {} have been finished.".format(epoch + 1))


    #After each generation is calculated, calculate the correlation coefficient and other indicators of each graph in the verification period, and take the mean value of all verification graphs as the result accuracy of the model after n generation training
    #Too slow! Not necessary to be used here
    #print("Size of real_valid",real_valid.size())
    #print("Size of validation",validation.size())
    # Generate noise for validation with G
    #noise_valid = torch.randn(valid_size, nz, 1, 1, device=device)  #[valid_size, channel, height, width]
    # Generate fake image of validation with G
    #with torch.no_grad():
    #    validation = netG(noise_valid, valid_label).detach().cpu()

    #Creating validation figs for each epoch with fixed_noise
    #plt.figure(figsize=(100,100))
    #plt.axis("off")
    #plt.title("Validation Images Epoch-{}".format(epoch + 1))
    #validation = utils.make_grid(validation, nrow=nrows)
    #plt.imshow(validation.permute(1, 2, 0)*0.5+0.5)
    #plt.savefig('validation-{}.jpg'.format(epoch + 1), bbox_inches='tight')
    #print("Validation figs of epoch number {} with fixed_noise has been saved.".format(epoch + 1))


    #with torch.no_grad():
    #    fake = netG(fixed_noise, fixed_prior).detach().cpu()
    #img_list.append(utils.make_grid(fake, nrow=nrows))
    #img_list is one (batch_size/nrows)*nrows 's group pictures

    #print("fake number",len(fake))
    #print("epoch",epoch, " finished")
    # Plot the fake images for this epoch
    #plt.figure(figsize=(100,100))
    #plt.axis("off")
    #plt.title("Fake Images Epoch-{}".format(epoch + 1))
    #fake = utils.make_grid(fake, nrow=nrows)
    #print("Train of epoch:",epoch + 1, "has finished.")
    #plt.imshow(fake.permute(1, 2, 0)*0.5+0.5)
    #plt.savefig('epoch-{}.jpg'.format(epoch + 1), bbox_inches='tight')
    #print("Fake figs of epoch number {} with fixed_noise has been saved.".format(epoch + 1))
print("All epoch trains have been finished.")
torch.save(netG.state_dict(), 'model-final.pt')

Starting Training Loop...(This may take a long time)
5.062432765960693
0.17813560366630554
[1/50][1/30]	Loss_D: 5.2406	Loss_G: 0.4327	D(x): 0.0155	D(G(z)): 0.1265 / 0.7600 	UsedTime:8.34 s
0.0
99.15345001220703
0.0
99.14234924316406
0.0
93.560791015625
0.0
79.06424713134766
[1/50][5/30]	Loss_D: 79.0642	Loss_G: 13.9153	D(x): 1.0000	D(G(z)): 0.9126 / 0.6048 	UsedTime:38.61 s
0.0001368977245874703
60.87874984741211
2.905644655227661
55.72026824951172
0.0
61.31035232543945
3.8506820487782534e-07
77.98846435546875
[1/50][9/30]	Loss_D: 77.9885	Loss_G: 11.4422	D(x): 1.0000	D(G(z)): 0.8831 / 0.6095 	UsedTime:68.49 s
4.3035777252953267e-07
68.23947143554688
9.352745109936222e-06
55.1064453125
1.1559027433395386
49.60528564453125
0.0062720258720219135
56.642513275146484
[1/50][13/30]	Loss_D: 56.6488	Loss_G: 13.8203	D(x): 0.9958	D(G(z)): 0.6357 / 0.5088 	UsedTime:98.14 s
0.0235298965126276
59.63194274902344
0.4424731433391571
40.88477325439453
5.939730644226074
35.985652923583984
7.72027942730346

In [ ]:
plt.title("Plotting 1-D array")
plt.xlabel("X axis")
plt.ylabel("Y axis")
x = np.array(range(0, 3))
plt.plot(x, D_losses, color="red", marker="o", label="Array elements")
plt.legend()
plt.show()


In [ ]:
plt.title("Plotting 1-D array")
plt.xlabel("X axis")
plt.ylabel("Y axis")
x = np.array(range(0, 3))
plt.plot(x, G_losses, color="red", marker="o", label="Array elements")
plt.legend()
plt.show()

# TEST

In [ ]:

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.image = nn.Sequential(
            # input is nz, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
        )
        self.prior = nn.Sequential(
            # state size. 1 x 1 x 1
            nn.ConvTranspose2d(1, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True)
            # state size. (ngf*8) x 4 x 4
        )
        self.main = nn.Sequential(
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d( ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf * 2) x 32 x 32
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 128 x 128
        )

    def forward(self, image, prior):
        image = self.image(image)
        prior = self.prior(prior)
        #Stitch two tensors (image, prior) together, dim=1 means splicing according to dimension 1 (column), that is, side by side
        incat = torch.cat((image, prior), dim=1)
        return self.main(incat)

In [ ]:

#Define the image mean correlation index calculation function in the verification period
def corrcoef(x, y):  #x,y are two mean arraies.
    n = len(x)
    #print("len(x)",n)
    # calculate sum
    sum1 = sum(x)
    #print("sum(x)",sum1)
    sum2 = sum(y)
    #print("sum(y)",sum2)
    sum3 = 0    #sum of squared errors
    sum4 = 0    #sum of absolute values of errors
    sum5 = 0    #sum of absolute percentage errors
    # calculate the sum of products
    multiplyofxy = np.multiply(x, y)
    sumofxy = 0
    for m in range (n):
       sumofxy = sumofxy + multiplyofxy [m]
       #Calculate the sum of squared errors
       sum3 = sum3 + (x[m]-y[m])*(x[m]-y[m])
       sum4 = sum4 + abs(x[m]-y[m])
       sum5 = sum5 + abs((x[m]-y[m])/x[m])
    #print("multiply(x, y)",sumofxy)
    #print("sum3=",sum3,"sum4=",sum4)
    rmse = sqrt(sum3/n)
    mae = sum4/n
    #print("rmse=",rmse,"mae=",mae)
    mape  = sum5/n *100  # Unit is %
    #print("mape=",mape)
    # 求=Calculate the sum of squares
    sumofx2 = sum([pow(i, 2) for i in x])
    sumofy2 = sum([pow(j, 2) for j in y])
    num = sumofxy - (float(sum1) * float(sum2) / n)
    # Calculating the Pearson Correlation Coefficient
    den = sqrt((sumofx2 - float(sum1 ** 2) / n) * (sumofy2 - float(sum2 ** 2) / n))
    #print("num=",num)
    #print("den=",den)
    #print("num / den=",num / den)
    return num / den, rmse, mae, mape
    # Returns the Pearson correlation coefficient, root mean square error (RMSE), mean absolute error (MAE), mean absolute percentage error (MAPE) in turn


In [ ]:

#Define tensor correlation coefficient calculation function
def tensor_corrcoef(a, b):   #a,b are two tensor  (num, band, X, Y)
    time1 = time.time()
    num_pic = a.size(0)     #num
    m = a.size(2)       #The value of image dimension 1
    n = a.size(3)       #The value of image dimension 2
    #print("num_pic m n are",num_pic, m, n)
    r2 = 0
    rmse = 0
    mae = 0
    mape = 0
    for p in range (num_pic):
        x = [0 for w in range(m*n)]
        y = [0 for w in range(m*n)]
        t = 0
        for q in range (m):
            for r in range (n):
                x[t] = (a[p][0][q][r].item() * 0.5 + 0.5)*100 - 50  # to be centigrade
                y[t] = (b[p][0][q][r].item() * 0.5 + 0.5)*100 - 50
                t = t + 1
        if (p + 1) % 100 == 0:
            time2 = time.time()
            print("Accounting r2 for validation pics {} have been finished ".format(p + 1))
            print("Time used", round(time2 - time1,1), "s")
            print("Estimated time remaining:", round((time2 - time1) / (p + 1) * (num_pic - p + 1), 1), "s")
    #print("x is",len(x), x)
    #print("y is",len(y), y)
    s = len(x)
    # calculate sum
    sum1 = sum(x)
    #print("sum(x)",sum1)
    sum2 = sum(y)
    #print("sum(y)",sum2)
    # calculate the sum of products
    multiplyofxy = np.multiply(x, y)
    sumofxy = 0
    sum3 = 0
    sum4 = 0
    sum5 = 0
    for r in range (s):
        sumofxy = sumofxy + multiplyofxy [r]
        sum3 = sum3 + (x[r]-y[r])*(x[r]-y[r])
        sum4 = sum4 + abs(x[r]-y[r])
        sum5 = sum5 + abs((x[r]-y[r])/x[r])
    #print("multiply(x, y)",sumofxy)
    rmse = sqrt(sum3/s)
    mae = sum4/s
    #print("rmse=",rmse,"mae=",mae)
    mape  = sum5/s *100  # Unit is %
    # Calculate the sum of squares
    sumofx2 = sum([pow(i, 2) for i in x])
    sumofy2 = sum([pow(j, 2) for j in y])
    num = sumofxy - (float(sum1) * float(sum2) / s)
    # Calculating the Pearson Correlation Coefficient
    den = sqrt((sumofx2 - float(sum1 ** 2) / s) * (sumofy2 - float(sum2 ** 2) / s))
    #print("num=",num)
    #print("den=",den)

    r2 = num / den
    return r2, rmse, mae, mape



In [ ]:

def tensor_corrcoef1(a, b):   #a,b为两个tensor  (num, band, X, Y)
    time1 = time.time()
    num_pic = a.size(0)     #num
    m = a.size(2)       #The value of image dimension 1
    n = a.size(3)       #The value of image dimension 2
    #print("num_pic m n are",num_pic, m, n)
    r2 = 0
    rmse = 0
    mae = 0
    mape = 0
    for p in range (num_pic):
        for q in range (m):
            for r in range (n):
                a[p][0][q][r] = (a[p][0][q][r] * 0.5 + 0.5)*100 - 50  # to be centigrade
                b[p][0][q][r] = (b[p][0][q][r] * 0.5 + 0.5)*100 - 50
        if (p + 1) % 100 == 0:
            time2 = time.time()
            print("Changes to be centigrade for validation pics {} have been finished ".format(p + 1))
            print("Time used", round(time2 - time1,1), "s")
            print("Estimated time remaining:", round((time2 - time1) / (p + 1) * (num_pic - p + 1), 1), "s")
    # Calculate the sum
    sum_a = tensor.sum(a)   #The sum of all numbers in tensor a
    sum1 = sum_a.item()
    print("sum(a)",sum1)
    sum_b = tensor.sum(b)   #The sum of all numbers in tensor b
    sum2 = sum_b.item()
    print("sum(b)",sum2)
    # Calculate the sum of products
    multiplyofxy = np.multiply(x, y)
    sumofxy = 0
    sum3 = 0
    sum4 = 0
    sum5 = 0
    for r in range (s):
        sumofxy = sumofxy + multiplyofxy [r]
        sum3 = sum3 + (x[r]-y[r])*(x[r]-y[r])
        sum4 = sum4 + abs(x[r]-y[r])
        sum5 = sum5 + abs((x[r]-y[r])/x[r])
    #print("multiply(x, y)",sumofxy)
    rmse = sqrt(sum3/s)
    mae = sum4/s
    #print("rmse=",rmse,"mae=",mae)
    mape  = sum5/s *100  # Unit is %
    # Calculate the sum of squares
    sumofx2 = sum([pow(i, 2) for i in x])
    sumofy2 = sum([pow(j, 2) for j in y])
    num = sumofxy - (float(sum1) * float(sum2) / s)
    # Calculating the Pearson Correlation Coefficient
    den = sqrt((sumofx2 - float(sum1 ** 2) / s) * (sumofy2 - float(sum2 ** 2) / s))
    #print("num=",num)
    #print("den=",den)

    r2 = num / den
    return r2, rmse, mae, mape





In [ ]:

if __name__ == '__main__':
  image_size = 128
  #workers = 2#8
  ngpu = 1
  #nz = 10
  ngf = 64
  nc = 1
  data_valid = "/content/prova/test"   #Need Change :The location of the test set data

  dataset_valid = dset.ImageFolder(root=data_valid,
                             transform=transforms.Compose([
                               transforms.Grayscale(num_output_channels=1),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5), (0.5)),
                             ]))
  print(f'Total Size of Dataset for Test: {len(dataset_valid)}')
  valid_num = len(dataset_valid)
  dataloader_valid = torch.utils.data.DataLoader(dataset=dataset_valid, batch_size=valid_num, shuffle=True, num_workers=workers)
  device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
  print("device to run on is", device)

In [ ]:


############################
# Read the Validation data
############################
valid_label = []                              #Validation set mean matrix Tensor
for i, data_valid in enumerate(dataloader_valid, 0):
    real_valid = data_valid[0].to(device)
valid_size = len(real_valid)
valid_mean = [0 for i in range(valid_size)]           #1D array of means for the validation set
valid_label_zero = torch.zeros(valid_num, 1, 1, 1).to(device)   #First generate valid_num Tensors of 1*1*1, all values are 0 (the verification data is only used for the generator, so it is 1*1*1 instead of 1*128*128)
for m in range(valid_num):
    #print("m =", m)
    ones_labePrior = torch.ones(1, 1).to(device)
    Avg = torch.mean(real_valid[m])
    #print("Avg =", Avg)
    valid_label_zero[m][0]= ones_labePrior * Avg
    Avg_num = Avg.cpu()
    Avg_num = Avg_num.item()
    #print("Avg_num=", Avg_num)
    valid_mean [m] = Avg_num
    #print("valid_mean [m]=", valid_mean [m])
valid_label = valid_label_zero.to(device)
print("The tensor of mean values of validation data have been created.")

############################
# Validation
############################

# Set random seed for reproducibility
# The same manualSeed as that used for training is generally used here
manualSeed = 512
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)
#For the same manualSeed, the following torch.rand numbers are fixed
noise_valid = torch.randn(valid_size, nz, 1, 1, device=device)  #[valid_size, channel, height, width]
netG = Generator(0).to(device)
netG.load_state_dict(torch.load('model-final.pt', map_location=device))
netG.eval()
# Generate the Fake Images
with torch.no_grad():
    validation = netG(noise_valid, valid_label).detach().cpu()
torch.cuda.empty_cache()
valid_list = torch.zeros(valid_size,image_size,image_size)
model_valid_mean = [0 for i in range(valid_size)]
valid_mean_centigrade = [0 for i in range(valid_size)]
model_valid_mean_centigrade = [0 for i in range(valid_size)]
for v in range (valid_size):
    valid_list[v] = validation [v][0]
    #print("valid_list[v] size", valid_list[v].size())          # valid_list[v] size torch.Size([128, 128])
    Avg1 = torch.mean(valid_list[v])
    Avg1 = Avg1.item()
    model_valid_mean [v] = Avg1
    #print("valid_list size", valid_list.size())                    # valid_list size torch.Size([3654, 128, 128])
    #print("valid_mean", valid_mean)
    #print("model_valid_mean", model_valid_mean)
for v in range (valid_size):
    valid_mean_centigrade [v] = (valid_mean [v] * 0.5 + 0.5)*100 - 50   # (* 0.5 + 0.5) changes to 0-1 and (*100 - 50) changes to real centigrade
    model_valid_mean_centigrade[v] = (model_valid_mean [v] * 0.5 + 0.5)*100 - 50
Pearson_correlation_coefficient_temp, RMSE_temp, MAE_temp, MAPE_temp = corrcoef(valid_mean_centigrade, model_valid_mean_centigrade)
print("Pearson correlation coefficient for mean value is {:.4f}".format(Pearson_correlation_coefficient_temp))
print("Root Mean Square Error(RMSE) for mean value is {:.2f}".format(RMSE_temp))
print("Mean Absolute Error(MAE) for mean value is {:.2f}".format(MAE_temp))
print("Mean Absolute Percentage Rrror(MAPE) for mean value is {:.2f}%".format(MAPE_temp))
output = open('result accuracy.xls','w',encoding='UTF-8')  #No need to create an excel table in advance, it will be automatically generated, gbk is the encoding method, supports Chinese, w stands for write
output.write('r2\t')
output.write(str(Pearson_correlation_coefficient_temp))
output.write('\n')
output.write('RMSE\t')
output.write(str(RMSE_temp))
output.write('\n')
output.write('MAE\t')
output.write(str(MAE_temp))
output.write('\n')
output.write('MAPE\t')
output.write(str(MAPE_temp))


# Run uncertainty analysis (change ManualSeedNumber)
ManualSeedNumber = 512
Pearson_correlation_coefficient_list = []
RMSE_list = []
MAE_list = []
MAPE_list = []
Seed_list = []
validation_list = torch.zeros(ManualSeedNumber, valid_size, nc, image_size,image_size)
print(validation_list.shape)
for i in range(ManualSeedNumber):
  manualSeed = i + 1
  Seed_list.append(manualSeed)
  print("Random Seed: ", manualSeed)
  random.seed(manualSeed)
  torch.manual_seed(manualSeed)
  torch.cuda.manual_seed(manualSeed)
  noise_valid = torch.randn(valid_size, nz, 1, 1, device=device)  #[valid_size, channel, height, width]
  netG = Generator(0).to(device)
  netG.load_state_dict(torch.load('model-final.pt', map_location=device))
  netG.eval()
  # Generate the Fake Images
  with torch.no_grad():
      validation = netG(noise_valid, valid_label).detach().cpu()
  torch.cuda.empty_cache()
  valid_list = torch.zeros(valid_size,image_size,image_size)
  model_valid_mean = [0 for i in range(valid_size)]
  valid_mean_centigrade = [0 for i in range(valid_size)]
  model_valid_mean_centigrade = [0 for i in range(valid_size)]
  for v in range (valid_size):
    valid_list[v] = validation [v][0]
    #print("valid_list[v] size", valid_list[v].size())          # valid_list[v] size torch.Size([128, 128])
    Avg1 = torch.mean(valid_list[v])
    Avg1 = Avg1.item()
    model_valid_mean [v] = Avg1
    #print("valid_list size", valid_list.size())                    # valid_list size torch.Size([3654, 128, 128])
    #print("valid_mean", valid_mean)
    #print("model_valid_mean", model_valid_mean)
  for v in range (valid_size):
    valid_mean_centigrade [v] = (valid_mean [v] * 0.5 + 0.5)*100 - 50
    model_valid_mean_centigrade[v] = (model_valid_mean [v] * 0.5 + 0.5)*100 - 50
  Pearson_correlation_coefficient_temp, RMSE_temp, MAE_temp, MAPE_temp = corrcoef(valid_mean_centigrade, model_valid_mean_centigrade)
  print("Pearson correlation coefficient for mean value is {:.4f}".format(Pearson_correlation_coefficient_temp))
  print("Root Mean Square Error(RMSE) for mean value is {:.2f}".format(RMSE_temp))
  print("Mean Absolute Error(MAE) for mean value is {:.2f}".format(MAE_temp))
  print("Mean Absolute Percentage Rrror(MAPE) for mean value is {:.2f}%".format(MAPE_temp))
  Pearson_correlation_coefficient_list.append(Pearson_correlation_coefficient_temp)
  RMSE_list.append(RMSE_temp)
  MAE_list.append(MAE_temp)
  MAPE_list.append(MAPE_temp)

  # Test the similarity between modeling images and real images in test set.
  finished = 0
  #finished = 1
  if finished == 1:
    print("Start calculating the similarity between training set (modeling) images and test set images.")
    print("This will seriously kill your time. It is recommended to not define the variable of 'finished' above as '1' to turn these commands on only if it is the final run!!!")
    real_valid.cpu()
    #Tensor_mean_r2, Tensor_mean_rmse, Tensor_mean_mae, Tensor_mean_mape = tensor_corrcoef(real_valid, validation)
    Tensor_mean_r2, Tensor_mean_rmse, Tensor_mean_mae, Tensor_mean_mape = tensor_corrcoef1(real_valid, validation)
    print("Tensor r2 = {:.4f}".format(Tensor_mean_r2), "Tensor RMSE = {:.2f}".format(Tensor_mean_rmse),"Tensor MAE ={:.2f}".format(Tensor_mean_mae),"Tensor MAPE ={:.2f}%".format(Tensor_mean_mape))

  validation_list[i] = validation
  i = i + 1

list_len = len(RMSE_list)
output = open('result accuracy uncertainty.xls','w',encoding='UTF-8')
output.write('Random Seed\tPearson_correlation_coefficient\tRMSE\tMAE\tMAPE\n')
for i in range(list_len):
    output.write(str(Seed_list[i]))
    output.write('\t')
    output.write(str(Pearson_correlation_coefficient_list[i]))    #The write function cannot write parameters of type int, so use str() to convert
    output.write('\t')                #Equivalent to Tab, change a cell
    output.write(str(RMSE_list[i]))
    output.write('\t')
    output.write(str(MAE_list[i]))
    output.write('\t')
    output.write(str(MAPE_list[i]))
    output.write('\t')
    output.write('\n')       #Write a new line immediately after writing a line
output.close()

TensorResultMean = torch.mean(validation_list, dim=0)
print(TensorResultMean.shape)
TensorResultStd = torch.std(validation_list, dim=0)
print(TensorResultStd.shape)
print("finished")

The tensor of mean values of validation data have been created.
Random Seed:  20


<ipython-input-60-d84b850f2e4a>:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netG.load_state_dict(torch.load('model-final.pt', map_location=device))


Pearson correlation coefficient for mean value is 0.3317
Root Mean Square Error(RMSE) for mean value is 29.75
Mean Absolute Error(MAE) for mean value is 29.75
Mean Absolute Percentage Rrror(MAPE) for mean value is 101.48%
torch.Size([512, 10, 1, 128, 128])
Random Seed:  1


<ipython-input-60-d84b850f2e4a>:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netG.load_state_dict(torch.load('model-final.pt', map_location=device))


Pearson correlation coefficient for mean value is 0.2842
Root Mean Square Error(RMSE) for mean value is 29.75
Mean Absolute Error(MAE) for mean value is 29.75
Mean Absolute Percentage Rrror(MAPE) for mean value is 101.48%
Random Seed:  2
Pearson correlation coefficient for mean value is -0.5457
Root Mean Square Error(RMSE) for mean value is 29.75
Mean Absolute Error(MAE) for mean value is 29.75
Mean Absolute Percentage Rrror(MAPE) for mean value is 101.48%
Random Seed:  3
Pearson correlation coefficient for mean value is 0.3081
Root Mean Square Error(RMSE) for mean value is 29.75
Mean Absolute Error(MAE) for mean value is 29.75
Mean Absolute Percentage Rrror(MAPE) for mean value is 101.47%
Random Seed:  4
Pearson correlation coefficient for mean value is 0.2775
Root Mean Square Error(RMSE) for mean value is 29.75
Mean Absolute Error(MAE) for mean value is 29.75
Mean Absolute Percentage Rrror(MAPE) for mean value is 101.48%
Random Seed:  5
Pearson correlation coefficient for mean value 